Name: Md Abdul Kadir

Linear Model


In [2]:
#Imports
import pandas as pd
import numpy as np

In [3]:
# Importan Methodes
# Replace the nan vale with mean, median, or most frequent value
def replaceNaN(dataframe, replaceby = "mean"):
    if (replaceby == "mean"):
        return dataframe.fillna(dataframe.mean())
    
    elif (replaceby == "median"):
        return dataframe.fillna(dataframe.median())
    
    elif (replaceby == "mostfrq"):
        return dataframe.fillna(dataframe.mode().iloc[0])
    
    else:return dataframe.dropna()
    
# Replce any nonnumaric char from a column.    
def replce_char(dataframe,collumn, char=r'^<'):
    dataframe[collumn] =  dataframe[collumn].replace(regex=char, value='')
    dataframe[collumn] = pd.to_numeric(dataframe[collumn],errors='coerce')
    return dataframe
    
    
    
# Scale the data using standard scaler or max min scaler. Menstion the names to scale. 
def scale(dataframe, coulumns, scaler="ss"):
    if (scaler == "mm"):
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        dataframe[coulumns] = scaler.fit_transform(dataframe[coulumns])
        return dataframe
    elif (scaler == "ss"):
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        datqframe[coulumns] = scaler.fit_transform(dataframe[coulumns])
        return dataframe
    
    else:return dataframe
        
    

    

In [4]:
#Load Dtasets

# Load Food Supply Data

Food_Supply_DF = pd.read_csv("../Data/updated_07_06_2020/Food_Supply_Quantity_kg_Data.csv")
Food_Supply_DF = replaceNaN(Food_Supply_DF, replaceby="mostfrq")



In [5]:
##Standard Scaler 
NOT_IPMORTANT_COLUMNS = ["Country", 'Unit (all except Population)', 'Population']

clm = Food_Supply_DF.columns.drop(NOT_IPMORTANT_COLUMNS)
print(clm)

Food_Supply_DF = replce_char(Food_Supply_DF,"Undernourished", char=r'^<')
scaled_df = scale(Food_Supply_DF, clm, scaler="mm")


Index(['Alcoholic Beverages', 'Animal fats', 'Animal Products',
       'Aquatic Products, Other', 'Cereals - Excluding Beer', 'Eggs',
       'Fish, Seafood', 'Fruits - Excluding Wine', 'Meat',
       'Milk - Excluding Butter', 'Miscellaneous', 'Offals', 'Oilcrops',
       'Pulses', 'Spices', 'Starchy Roots', 'Stimulants', 'Sugar & Sweeteners',
       'Sugar Crops', 'Treenuts', 'Vegetable Oils', 'Vegetables',
       'Vegetal Products', 'Obesity', 'Undernourished', 'Confirmed', 'Deaths',
       'Recovered', 'Active'],
      dtype='object')


In [6]:
# Randomise data
scaled_df = scaled_df.sample(frac=1,random_state=1).reset_index(drop=True)
# scaled_df

# A simple regression model for 1 variable.

In [7]:
# Chosse training input and output
#XY = scaled_df[['Undernourished','Obesity' ]]
#XY.plot.scatter(x = 'Undernourished', y ='Obesity')

FEATURE_CLS = ['Obesity', 'Undernourished', 'Deaths','Recovered','Active','Confirmed']
FEATURE = ['Obesity', 'Undernourished', 'Deaths','Recovered' ]

ALL_OUTPUT = ['Alcoholic Beverages', 'Animal Products', 'Animal fats',
       'Aquatic Products, Other', 'Cereals - Excluding Beer', 'Eggs',
       'Fish, Seafood', 'Fruits - Excluding Wine', 'Meat',
       'Milk - Excluding Butter', 'Offals', 'Oilcrops', 'Pulses', 'Spices',
       'Starchy Roots', 'Stimulants', 'Sugar Crops', 'Sugar & Sweeteners',
       'Treenuts', 'Vegetal Products', 'Vegetable Oils', 'Vegetables',
       'Miscellaneous']
GOOD_OUTPUT =['Animal Products', 'Animal fats','Vegetal Products','Vegetable Oils' ]

In [8]:
# Devide trian and test set
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(scaled_df, test_size=0.2, random_state=42)

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold 
X = pd.DataFrame(scaled_df[FEATURE_CLS])
y = pd.DataFrame(scaled_df[GOOD_OUTPUT])
model = LinearRegression()
scores = []
kfold = KFold(n_splits=3, shuffle=True, random_state=1)
for i, (train, test) in enumerate(kfold.split(X, y)):
    model.fit(X.iloc[train,:], y.iloc[train,:])
    score = model.score(X.iloc[test,:], y.iloc[test,:])
    scores.append(score)
print(scores)

[0.23920421662986885, -3.1786696492969977, 0.2175182510186099]


In [10]:
from sklearn.linear_model import LinearRegression
def Regression(features, response, data, method = LinearRegression()):
    X = data[features]
    Y = data[response]
    return model.fit(X, Y)
    
    
    
def TestScore(features, response, data, model):
    print('Output: ', response)
    X = data[features]
    Y = data[response]
    return model.score(X,Y)
    
def PlotRegression(features, response, data, model):
    import matplotlib.pyplot as plt
    X = data[features]
    Y = data[response]
    y_pred = model.predict(X)
    plt.scatter(X, Y, s=10)
    plt.plot(X, y_pred, color='r')
    plt.show()
    

In [11]:
from sklearn import linear_model
model = Regression(FEATURE_CLS, GOOD_OUTPUT, train_set, method = linear_model.Lasso(alpha=0.1) )


In [12]:
TestScore(FEATURE_CLS, GOOD_OUTPUT,test_set, model)

Output:  ['Animal Products', 'Animal fats', 'Vegetal Products', 'Vegetable Oils']


-3.6181737475778593

In [13]:
#PlotRegression(FEATURE_CLS, ALL_OUTPUT  ,tra in_set, model)

In [14]:
for cls in ALL_OUTPUT:
    model = Regression(FEATURE, cls, train_set, method = linear_model.Lasso(alpha=0.5) )
    print('R^2 : ', TestScore(FEATURE, cls,test_set, model))
    #PlotRegression(FEATURE, cls ,test_set, model)

Output:  Alcoholic Beverages
R^2 :  0.012966420549500146
Output:  Animal Products
R^2 :  0.3266763032232859
Output:  Animal fats
R^2 :  0.10012787125844702
Output:  Aquatic Products, Other
R^2 :  -46.732919591801796
Output:  Cereals - Excluding Beer
R^2 :  0.3306840033819963
Output:  Eggs
R^2 :  0.2878016465935942
Output:  Fish, Seafood
R^2 :  -0.1800763222310393
Output:  Fruits - Excluding Wine
R^2 :  -0.0011452530491427027
Output:  Meat
R^2 :  0.1493667755650725
Output:  Milk - Excluding Butter
R^2 :  0.17457727080602814
Output:  Offals
R^2 :  0.019803438466795664
Output:  Oilcrops
R^2 :  -0.7777524802263442
Output:  Pulses
R^2 :  0.2348711130733202
Output:  Spices
R^2 :  0.04288068023189051
Output:  Starchy Roots
R^2 :  0.1524583645950105
Output:  Stimulants
R^2 :  -0.06617865551850333
Output:  Sugar Crops
R^2 :  -0.006897764958550434
Output:  Sugar & Sweeteners
R^2 :  0.09612198836944597
Output:  Treenuts
R^2 :  0.016051722632956178
Output:  Vegetal Products
R^2 :  0.32672663441316

In [15]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

from sklearn import linear_model
for cls in ALL_OUTPUT: 
    poly_transformer = PolynomialFeatures(1) 
    X_train_transformed = poly_transformer.fit_transform(train_set[FEATURE])
    X_test_transformed = poly_transformer.fit_transform(test_set[FEATURE])
    #print(X_train_transformed)
    model = LinearRegression()
    model.fit(X_train_transformed, train_set[cls])
    score = model.score(X_test_transformed, test_set[cls])
    print('R^2 Score of ' , cls ,' : ' , score)
    
    scores = []
    kfold = KFold(n_splits=3, shuffle=True, random_state=1)
    for i, (train, test) in enumerate(kfold.split(X, y)):
        model.fit(X.iloc[train,:], y.iloc[train,:])
        score = model.score(X.iloc[test,:], y.iloc[test,:])
        scores.append(score)
    print(scores)
    
    
    
    

R^2 Score of  Alcoholic Beverages  :  0.012966420549500146
[0.23920421662986885, -3.1786696492969977, 0.2175182510186099]
R^2 Score of  Animal Products  :  0.3266763032232859
[0.23920421662986885, -3.1786696492969977, 0.2175182510186099]
R^2 Score of  Animal fats  :  0.10012787125844724
[0.23920421662986885, -3.1786696492969977, 0.2175182510186099]
R^2 Score of  Aquatic Products, Other  :  -46.732919591801824
[0.23920421662986885, -3.1786696492969977, 0.2175182510186099]
R^2 Score of  Cereals - Excluding Beer  :  0.3306840033819962
[0.23920421662986885, -3.1786696492969977, 0.2175182510186099]
R^2 Score of  Eggs  :  0.2878016465935943
[0.23920421662986885, -3.1786696492969977, 0.2175182510186099]
R^2 Score of  Fish, Seafood  :  -0.18007632223103953
[0.23920421662986885, -3.1786696492969977, 0.2175182510186099]
R^2 Score of  Fruits - Excluding Wine  :  -0.0011452530491429247
[0.23920421662986885, -3.1786696492969977, 0.2175182510186099]
R^2 Score of  Meat  :  0.14936677556507227
[0.2392

In [16]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=2)
for cls in ALL_OUTPUT: 
    poly_transformer = PolynomialFeatures(1) 
    X_train_transformed = poly_transformer.fit_transform(train_set[FEATURE_CLS])
    X_test_transformed = poly_transformer.fit_transform(test_set[FEATURE_CLS])
    #print(X_train_transformed)
    model.fit(X_train_transformed, train_set[cls])
    score = model.score(X_test_transformed, test_set[cls])
    print('R^2 Score of ' , cls ,' : ' , score)
    
    
    

R^2 Score of  Alcoholic Beverages  :  0.03965386084765743
R^2 Score of  Animal Products  :  0.2818409678086853
R^2 Score of  Animal fats  :  0.2488966501554527
R^2 Score of  Aquatic Products, Other  :  -1.6823712118679053
R^2 Score of  Cereals - Excluding Beer  :  0.17468054968955704
R^2 Score of  Eggs  :  0.19160947741222967
R^2 Score of  Fish, Seafood  :  -0.30056953540426146
R^2 Score of  Fruits - Excluding Wine  :  -0.37285329753555474
R^2 Score of  Meat  :  -0.1127824832750921
R^2 Score of  Milk - Excluding Butter  :  0.17327533884201818
R^2 Score of  Offals  :  -0.09694573834612319
R^2 Score of  Oilcrops  :  -6.048387267544269
R^2 Score of  Pulses  :  0.23384268393426155
R^2 Score of  Spices  :  -0.08779893905487501
R^2 Score of  Starchy Roots  :  -1.012288900161058
R^2 Score of  Stimulants  :  -0.035189707024323225
R^2 Score of  Sugar Crops  :  -0.30892385964715463
R^2 Score of  Sugar & Sweeteners  :  -0.05813924159332151
R^2 Score of  Treenuts  :  -0.38462477638180315
R^2 Score

In [17]:
from sklearn.svm import SVR

model =  SVR(C=1.0, epsilon=0.1)
for cls in ALL_OUTPUT: 
    poly_transformer = PolynomialFeatures(1) 
    X_train_transformed = poly_transformer.fit_transform(train_set[FEATURE_CLS])
    X_test_transformed = poly_transformer.fit_transform(test_set[FEATURE_CLS])
    #print(X_train_transformed)
    model.fit(X_train_transformed, train_set[cls])
    score = model.score(X_test_transformed, test_set[cls])
    print('R^2 Score of ' , cls ,' : ' , score)
    

R^2 Score of  Alcoholic Beverages  :  -0.056018164418486505
R^2 Score of  Animal Products  :  0.2228310191721451
R^2 Score of  Animal fats  :  0.15179297651639911
R^2 Score of  Aquatic Products, Other  :  -1019.5110111341505
R^2 Score of  Cereals - Excluding Beer  :  0.263271199417341
R^2 Score of  Eggs  :  0.21431862813213998
R^2 Score of  Fish, Seafood  :  -0.10423127050655112
R^2 Score of  Fruits - Excluding Wine  :  -0.18934970098517745
R^2 Score of  Meat  :  -0.0718318280926673
R^2 Score of  Milk - Excluding Butter  :  0.1770147902362431
R^2 Score of  Offals  :  -0.1489398333348575
R^2 Score of  Oilcrops  :  -0.4792396787348754
R^2 Score of  Pulses  :  0.11666434300419559
R^2 Score of  Spices  :  0.07353487553901461
R^2 Score of  Starchy Roots  :  0.19160087531171255
R^2 Score of  Stimulants  :  -0.05900146126968453
R^2 Score of  Sugar Crops  :  -0.18342301369347802
R^2 Score of  Sugar & Sweeteners  :  0.10790788923753825
R^2 Score of  Treenuts  :  -0.006830195896169844
R^2 Score 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(max_depth=2, random_state=0)
for cls in ALL_OUTPUT: 
    poly_transformer = PolynomialFeatures(1) 
    X_train_transformed = poly_transformer.fit_transform(train_set[FEATURE])
    X_test_transformed = poly_transformer.fit_transform(test_set[FEATURE])
    #print(X_train_transformed)
    model.fit(X_train_transformed, train_set[cls])
    score = model.score(X_test_transformed, test_set[cls])
    print('R^2 Score of ' , cls ,' : ' , score)
    

# Section: Dataset and model class implementation

In [19]:
from sklearn.model_selection import train_test_split

# dataset class
class NutritionData:
    
    def __init__(self, data_path):
        
        self.data_path = data_path
        dataframe = pd.read_csv(self.data_path)
        dataframe = self.__replaceNaN(dataframe, replaceby="mostfrq")
        
        ##Standard Scaler 
        NOT_IPMORTANT_COLUMNS = ["Country", 'Unit (all except Population)', 'Population']

        clm = dataframe.columns.drop(NOT_IPMORTANT_COLUMNS)

        dataframe = self.__replace_char(dataframe, "Undernourished", char=r'^<')
        scaled_df = self.__scale(dataframe, clm, scaler="mm")
        scaled_df = scaled_df.sample(frac=1,random_state=1).reset_index(drop=True)

        self.dataset = scaled_df
        
    
        # Importan Methodes
    # Replace the nan vale with mean, median, or most frequent value
    def __replaceNaN(self, dataframe, replaceby = "mean"):
        if (replaceby == "mean"):
            return dataframe.fillna(dataframe.mean())

        elif (replaceby == "median"):
            return dataframe.fillna(dataframe.median())

        elif (replaceby == "mostfrq"):
            return dataframe.fillna(dataframe.mode().iloc[0])

        else:return dataframe.dropna()
        
        

    # Replce any nonnumaric char from a column.    
    def __replace_char(self, dataframe, collumn, char=r'^<'):
        
        dataframe[collumn] =  dataframe[collumn].replace(regex=char, value='')
        dataframe[collumn] = pd.to_numeric(dataframe[collumn],errors='coerce')
        return dataframe



    # Scale the data using standard scaler or max min scaler. Menstion the names to scale. 
    def __scale(self, dataframe, coulumns, scaler="ss"):
        
        if (scaler == "mm"):
            from sklearn.preprocessing import MinMaxScaler
            scaler = MinMaxScaler()
            dataframe[coulumns] = scaler.fit_transform(dataframe[coulumns])
            return dataframe
        elif (scaler == "ss"):
            from sklearn.preprocessing import StandardScaler
            scaler = StandardScaler()
            datqframe[coulumns] = scaler.fit_transform(dataframe[coulumns])
            return dataframe

        else:return dataframe





In [20]:
import pickle
import logging
import os

In [140]:
# Generalizing class to test and train all the models
class NutritionModel:
    
    def __init__(self, dataset=None):
        self.dataset = dataset
        self.model = None
        self.train_set = None
        self.test_set = None
        
        
    def split_data(self, test_size=0.2, random_state=42):
        # splits the dataset in train and test set
        self.train_set, self.test_set  = train_test_split(self.dataset, test_size=test_size, random_state=42)
      
    
    def set_model(self, model):
        # setting up the model
        self.model = model
        
        
    def save_model(self, pkl_filename='model.pkl'):
        with open(pkl_filename, 'wb') as file:
            pickle.dump(self.model, file)
    
    
    
    def load_model(self, pkl_filename='model.pkl'):
        with open(pkl_filename, 'rb') as file:
            self.model = pickle.load(file)
    
        
    
    def train_model(self, x_column, y_column):
        #traing the model
        # input arg: x column name, y column names
        self.model.fit(self.train_set[x_column], self.train_set[y_column])
    
    
    def test_model(self, x_column, y_column): 
        score = self.model.score(self.test_set[x_column], self.test_set[y_column])
        return score
    
    
    def predict(self, X):
        return self.model.predict(X)
    

In [38]:
FEATURE_CLS = ['Obesity', 'Undernourished', 'Deaths','Recovered','Active','Confirmed']
FEATURE = ['Obesity', 'Undernourished', 'Deaths','Recovered' ]

ALL_OUTPUT = ['Alcoholic Beverages', 'Animal Products', 'Animal fats',
       'Aquatic Products, Other', 'Cereals - Excluding Beer', 'Eggs',
       'Fish, Seafood', 'Fruits - Excluding Wine', 'Meat',
       'Milk - Excluding Butter', 'Offals', 'Oilcrops', 'Pulses', 'Spices',
       'Starchy Roots', 'Stimulants', 'Sugar Crops', 'Sugar & Sweeteners',
       'Treenuts', 'Vegetal Products', 'Vegetable Oils', 'Vegetables',
       'Miscellaneous']
GOOD_OUTPUT=['Animal Products', 'Animal fats','Vegetal Products','Vegetable Oils' ]

In [39]:

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
print(rf.get_params())
#Parameters currently in use:


Parameters currently in use:

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [40]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)



# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1, refit=True)
# Fit the random search model

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [41]:
log_path = '../log/random_forest/'
data_root = '../Data/updated_04_07_2020/Food_Supply_Quantity_kg_Data.csv'

# change here
data_file = "Food_Supply_Quantity_kg_Data.csv"
log_file = log_path + 'log.txt'

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Print arguments
logger.info('Log file is %s.' % log_file)
# logger.info('Data path is %s.' % data_path)
# logger.info('Export path is %s.' % xp_path)

data_path = data_root + data_file


if not os.path.exists(data_file.split('.')[0]):
    model_dirs = log_path + data_file.split('.')[0]
    os.makedirs(model_dirs)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/mak/Uni-courses/SS2020/DS/Nutrition-and-immune-systems/log/random_forest/log.txt'

In [42]:
# model.save_model()
# model.load_model()
import csv
fieldnames = ['model_name', 'x_column', 'y_column', 'score']
with open('../log/Results_Food_Supply_Quantity_kg_Data.csv', 'w', newline='') as csvfile:
    
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

FileNotFoundError: [Errno 2] No such file or directory: '../log/Results_Food_Supply_Quantity_kg_Data.csv'

In [43]:
from sklearn import linear_model
data_path = "../Data/updated_07_06_2020/Food_Supply_Quantity_kg_Data.csv"
dataobj = NutritionData(data_path=data_path) # creating dataset

# creating model classes
model = NutritionModel(dataset=dataobj.dataset)

lm = linear_model.LinearRegression()

model.set_model(model=rf_random)
model.split_data()

with open('../log/Results_Food_Supply_Quantity_kg_Data.csv', 'a', newline='') as csvfile:
    for y_val in ALL_OUTPUT:
        x_val = FEATURE
        model.train_model(x_column=x_val, y_column=y_val)
        score = model.test_model(x_column=x_val, y_column=y_val)
        model.save_model(pkl_filename=os.path.join(model_dirs, y_val+'.pkl'))
        print(score)
        obj = csv.DictWriter(csvfile, fieldnames=fieldnames)
        obj.writerow({'model_name':'Random Forest', 'x_column': x_val, 'y_column': y_val, 'score': score})

FileNotFoundError: [Errno 2] No such file or directory: '../log/Results_Food_Supply_Quantity_kg_Data.csv'

In [45]:
# Tareque 

# Create class to that take a csv file as an input
# Return output prediction 

from sklearn import linear_model

dataobj = NutritionData(data_path=data_path) # creating dataset
# creating model classes
model = NutritionModel(dataset=dataobj.dataset)

model.set_model(model=rf_random)
model.split_data()

x_val = FEATURE
model.train_model(x_column=x_val, y_column=GOOD_OUTPUT)
score = model.test_model(x_column=x_val, y_column='')
#model.save_model(pkl_filename=os.path.join(, y_val+'.pkl'))
print(score)

        
        
    
    


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    7.4s finished


0.2385564018604872


In [114]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import RandomizedSearchCV

method_names = {}
# Adding linear regression
LinearREG = LinearRegression()
method_names["Linear Regression"] = LinearREG

# Adding reage regression 
params_Ridge = {'alpha': [1, 0.1,0.01,0.001,0.0001,0] ,
                "fit_intercept": [True, False],
                "solver": ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}


LinearREGRedgeCV = RandomizedSearchCV(estimator = Ridge(),
                                      param_distributions = params_Ridge,
                                      n_iter = 10,
                                      cv = 5, verbose=2,
                                      random_state=42,
                                      n_jobs = -1, refit=True)

method_names["Redge"] = LinearREGRedgeCV


# Adding Lasso regression

params_Lasso = {'alpha': [1, 0.1,0.01,0.001,0.0001,0] ,
                "fit_intercept": [True, False],
               }
LinearREGLassoCV =  RandomizedSearchCV(estimator = Lasso(),
                                      param_distributions = params_Lasso,
                                      n_iter = 10,
                                      cv = 5, verbose=2,
                                      random_state=42,
                                      n_jobs = -1, refit=True)



method_names["Lasso"] = LinearREGLassoCV


# Adding SVR


params_SVR = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

SupportVectorRegression =  RandomizedSearchCV(estimator = SVR(),
                                      param_distributions = params_SVR,
                                      n_iter = 10,
                                      cv = 5, verbose=2,
                                      random_state=42,
                                      n_jobs = -1, refit=True)



method_names["SVR"] = SupportVectorRegression


# Adding Decission Tree
params_decission_tree = [{'criterion': ['mse', 'friedman_mse', 'mae'],
                          'ccp_alpha': [0.0 , 1e-3, 1e-4, 0.5, 1.0],
                          'max_features' : [1,2,3,4,5],
                         'max_depth': [1,2,3,4,],
                         'random_state':[1,4,5,100]},
                         {'criterion': [ 'friedman_mse'],
                          'ccp_alpha': [1.0],
                          'max_features' : [1,2,3,4,5],
                         'max_depth': [10,11,12,13,14],
                         'random_state':[1,4,5,100]},
                         {'criterion': [ 'mae'],
                          'ccp_alpha': [0.0 , 1e-3, 1e-4, 0.5, 1.0],
                          'max_features' : [1,2,3,4,5],
                         'max_depth': [5,6,7,8,9],
                         'random_state':[100]},
                         
                         ]

#print(DecisionTreeRegressor().get_params().keys())

DecissionTree =  RandomizedSearchCV(estimator = DecisionTreeRegressor(),
                                      param_distributions = params_decission_tree,
                                      n_iter = 10,
                                      cv = 5, verbose=2,
                                      random_state=42,
                                      n_jobs = -1, refit=True)


method_names["DecissionTree"] = DecissionTree

#  Adding RandomForestRegressor


params_random_forest = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

RandomForest =  RandomizedSearchCV(estimator = RandomForestRegressor(),
                                      param_distributions = params_random_forest,
                                      n_iter = 10,
                                      cv = 5, verbose=2,
                                      random_state=42,
                                      n_jobs = -1, refit=True)


method_names["RandomForest"] = RandomForest




Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   14.2s finished


0.24156884522415123


In [126]:
# data_file: Adrres of csv
# methods : dictionary of methods (method_names) 
def search_models(data_file, methods, features, predictors):
    dataobj = NutritionData(data_path=data_file)
    model = NutritionModel(dataset=dataobj.dataset)
    model.split_data()
    x_val = features
    GOOD_OUTPUT = predictors
    r2_score = []
    for key, value in method_names.items():
        model.set_model(model=value)
        if key == 'SVR':
            score = 0
            for item in GOOD_OUTPUT:
                model.train_model(x_column=x_val, y_column=item)
                score += model.test_model(x_column=x_val, y_column=item)
                #print(score/len(GOOD_OUTPUT))
            r2_score.append(score/len(GOOD_OUTPUT))
            
        
        else:
            model.train_model(x_column=x_val, y_column=GOOD_OUTPUT)
            score = model.test_model(x_column=x_val, y_column=GOOD_OUTPUT)
            r2_score.append(score)
            #print(score)


        
    data_frame_dict = {'Methods': list(method_names.keys()),
                   'Input':[FEATURE for i in range(len(method_names.keys()))],
                   'Output':[GOOD_OUTPUT for i in range(len(method_names.keys()))], 
                   'R^2 Score': r2_score }

    df = pd.DataFrame(data=data_frame_dict)
    print(df.head)
    return df

    

In [130]:
# Choose the best model for each of four dataset

paths = [os.path.join("../Data/updated_07_06_2020", "Food_Supply_Quantity_kg_Data.csv" ),
        os.path.join("../Data/updated_07_06_2020", "Fat_Supply_Quantity_Data.csv" ),
        os.path.join("../Data/updated_07_06_2020", "Food_Supply_kcal_Data.csv" ),
        os.path.join("../Data/updated_07_06_2020", "Protein_Supply_Quantity_Data.csv" )]


for path in paths:
    protein_prdiction = search_models(path,method_names,FEATURE, GOOD_OUTPUT)
    output_path = os.path.join('../Data/outputfiles', os.path.split(path)[-1])
    protein_prdiction.to_csv(output_path)





Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits


...

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   14.2s finished


<bound method NDFrame.head of              Methods                                         Input  \
0  Linear Regression  [Obesity, Undernourished, Deaths, Recovered]   
1              Redge  [Obesity, Undernourished, Deaths, Recovered]   
2              Lasso  [Obesity, Undernourished, Deaths, Recovered]   
3                SVR  [Obesity, Undernourished, Deaths, Recovered]   
4      DecissionTree  [Obesity, Undernourished, Deaths, Recovered]   
5       RandomForest  [Obesity, Undernourished, Deaths, Recovered]   

                                              Output  R^2 Score  
0  [Animal Products, Animal fats, Vegetal Product...   0.245428  
1  [Animal Products, Animal fats, Vegetal Product...   0.236165  
2  [Animal Products, Animal fats, Vegetal Product...   0.244399  
3  [Animal Products, Animal fats, Vegetal Product...   0.211328  
4  [Animal Products, Animal fats, Vegetal Product...   0.171732  
5  [Animal Products, Animal fats, Vegetal Product...   0.239984  >


In [155]:

#Store the best model for all datasets

def store_models(data_file, model_name, features, predictors, output_dir):
    dataobj = NutritionData(data_path=data_file)
    model = NutritionModel(dataset=dataobj.dataset)
    model.split_data()
    x_val = features
    GOOD_OUTPUT = predictors
    model.set_model(model=value)
    model_file = os.path.join(output_dir, os.path.split(data_file)[-1].split('.')[0] + '.pkl')
    if key == 'SVR':
        score = 0
        for item in GOOD_OUTPUT:
            model.train_model(x_column=x_val, y_column=item)
            score += model.test_model(x_column=x_val, y_column=item)
            print(score/len(GOOD_OUTPUT))
            
            
        
    else:
        model.train_model(x_column=x_val, y_column=GOOD_OUTPUT)
        score = model.test_model(x_column=x_val, y_column=GOOD_OUTPUT)
        model.save_model(model_file)
        print(score)

    return (os.path.split(data_file)[-1].split('.')[0], model_file)





In [156]:
# Four models for four datasets
model_list =[]
for path in paths:
    model_address  = store_models(path,method_names["RandomForest"],FEATURE, GOOD_OUTPUT, '../Data/outputfiles/models')
    model_list.append(model_address)
    



Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   14.6s finished


0.24318575676273854
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   13.8s finished


0.2602929768880775
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   13.6s finished


0.30784424638319974
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   14.0s finished


0.234970050407205


In [169]:
# Prdict output

nutrition_category = []
food_consumtion = []


for model in model_list:

    nutrition_category.append(model[0])
    nutrition_model = NutritionModel()
    nutrition_model.load_model(model[1])
    food_consumtion.append(list(nutrition_model.predict([[0,0,0,1],])[0]))
    
print(food_consumtion)
    
df_dict = {'Nutrition Category': nutrition_category,
           str(FEATURE):[[0,0,0,1] for i in range(4)],
           str(GOOD_OUTPUT): food_consumtion
          }

df = pd.DataFrame(data= df_dict)
df.to_csv(os.path.join('../Data/outputfiles', 'ideal_food_consumtion.csv'))
    


[[0.4420617236793843, 0.13076530409540074, 0.5579144138971044, 0.3955578905017301], [0.5269269428219676, 0.23249715525730033, 0.4730760784670881, 0.389969916260956], [0.4672009965706361, 0.15359162755182085, 0.5326745960321688, 0.42232611664871184], [0.6413811485023138, 0.09672828808652183, 0.3584873705494411, 0.14035575012790616]]


In [152]:
a = [[1,3] *5]
print(a)

[[1, 3, 1, 3, 1, 3, 1, 3, 1, 3]]
